In [ ]:
# Name: example_basic
# Authors: Stephan Meighen-Berger, Kruteesh Desai
# An analysis script for Christian on P-ONE new effective areas

In [ ]:
# imports
from constants import pdm_constants as const
from config import config
from pone_aeff import Aeff
from dm2nu import DM2Nu
from atm_shower import Atm_Shower
from detectors import Detector
import numpy as np
import matplotlib.pyplot as plt
from limit_calc import Limits
from bkgrd_calc import Background
from signal_calc import Signal
from pdm import PDM
from scipy.interpolate import UnivariateSpline
from scipy.stats import chi2
from scipy.stats import *
from pone_aeff import Aeff
import pickle
import math
import pandas as pd
import csv
from matplotlib.lines import Line2D
from matplotlib import rc

In [ ]:
# picture path
PICS = '../pics/'
name=[r'$\nu_{\mu}$', r"$\nu_e$", r"$\nu_{\tau}$"]  # ,r'$\nu_{e}$',r'$\nu_{\tau}$'
# Plotting standards
std_size = 6.  # Size of the plot
fontsize = 15.  # Fontsize in the plot
lw=0.1  # Linewidth
h_length=0.5  # Handle length for the legends
export_dpi = 500  # Dpi for the image export
color_pone='#7570b3'
color_ice='#e7298a'
color_combined='#33a02c'
color_thermal='#1f78b4'
color_unitary='k'
color_4='#1b9e77'
color_6='#d95f02'
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)

In [ ]:
mass_grid = config["simulation parameters"]["mass grid"]
sv_grid = config["simulation parameters"]["sv grid"]

In [ ]:
hits = [2]
modules = [3]
spacing = [50, 80, 110, 140, 200]
pos_res = [2]

In [ ]:
pdm_res = {}
for i,sp in enumerate(spacing):
    config['general']['detector'] = 'POne'
    config['general']['pone type'] = 'new'
    config['pone']['smearing'] = 'smeared'
    config['pone_christian']['hit'] = hits[0]
    config['pone_christian']['module'] = modules[0]
    config['pone_christian']['spacing'] = sp
    config['pone_christian']['pos res'] = pos_res[0]
    pdm_t = PDM()
    pdm_res[hits[0], modules[0], sp, pos_res[0]] = pdm_t.results

In [ ]:
pickle.dump(pdm_res,open('../data/tmp_files/smeared_limits.pkl', 'wb'))


In [ ]:
# Loading comparison data
th_cl = np.loadtxt(open("../data/thermal_relic.csv"), delimiter = ",")
unitarity = np.loadtxt(open('../data/Unitarity_bound.csv'), delimiter = ',')
cl_lit = np.loadtxt('../data/CL_PONE.csv', delimiter=",")
cl_lit = (cl_lit[cl_lit[:, 0].argsort()])
unitarity = (unitarity[unitarity[:, 0].argsort()])

#limits_ice = pickle.load(open('/home/kruteesh/Desktop/dm_data_server/28_03_desk/data/limits_results_IceCube.pkl','rb'))
limits_pone_unsm = pickle.load(open("../data/28_03_02/unsmeared/limits_results_POne_unsm.pkl","rb"))
#limits_com = pickle.load(open("/home/kruteesh/Desktop/dm_data_server/14_03/data/limits_results.pkl", "rb"))
#limits_pone_unsm = pickle.load(open("../data/28_03_02/unsmeared/limits_results_POne_unsm.pkl","rb"))

limits_pone_45_25= pickle.load(open("../data/14_04/smeared/45_25/limits_results_POne_sm.pkl","rb"))

limits_pone_35_15= pickle.load(open("../data/14_04/smeared/35_15/limits_results_POne_sm.pkl","rb"))

limits_pone_25_10= pickle.load(open("../data/14_04/smeared/25_10/limits_results_POne_sm.pkl","rb"))

limits_pone_15_009= pickle.load(open("../data/14_04/smeared/15_009/limits_results_POne_sm.pkl","rb"))


cl_lit_ice = np.loadtxt('../data/IceCube_HE.csv', delimiter=",")
cl_lit_ice = (cl_lit_ice[cl_lit_ice[:, 0].argsort()])
xx,yy = np.meshgrid(np.log10( np.logspace(1, 6, 20)),np.log10( np.logspace(1, 6, 20)))

In [ ]:
limits_sm_christ = pickle.load(open('../data/tmp_files/smeared_limits.pkl', 'rb'))

In [ ]:
color = ['black', 'orange', 'purple']
color_sm = ['red', 'black', 'purple']
color_pa = ['']
color_pa = ['']
color_pa = ['']
color_pa = ['']

ll = ['-','--', '-.', 'dotted']
 # std_size, std_size * 6. / 8.),
for i in pdm_res.keys():
    # Limits
    # ---------------------------------------------
    # Setup
    # print(i, type(i))
    figure, (ax1) = plt.subplots(1, 1, figsize=(std_size,std_size), sharex=True) 
    # ---------------------------------------------
    # Plotting
    ax1.set_title('spacing =  %f' % (i[2]),fontsize=fontsize)  # +' Scaling Correction = '+str(config['advanced']['scaling correction'])

    ax1.contour(pdm_res[i]["numu"], levels = [0.9],  linestyles='-',
               origin='lower', extent=(min(np.log10(mass_grid)), max(np.log10(mass_grid)),
                                       min(np.log10(sv_grid)), max(np.log10(sv_grid))))
    ax1.contour(limits_sm_christ[i]["numu"], levels = [0.9],  linestyles='--',
               origin='lower', extent=(min(np.log10(mass_grid)), max(np.log10(mass_grid)),
                                       min(np.log10(sv_grid)), max(np.log10(sv_grid))))

    ax1.plot(np.log10(cl_lit[:, 0]), np.log10(cl_lit[:, 1]), 'g', label='literature P-ONE')
    #ax1.plot(np.log10(cl_lit_ice[:, 0]), np.log10(cl_lit_ice[:, 1]),'m',label='literature  IceCube')
    ax1.plot(np.log10(th_cl[:, 0]), np.log10(th_cl[:, 1]), 'b', 
             label='Thermal Relic Abundance')
    #ax1.plot(np.log10(unitarity[:, 0]), np.log10(unitarity[:, 1]), '.k', 
    #         label='Unitarity Bound')
    # ---------------------------------------------
    # Axis options
    ax1.set_xscale("linear")
    ax1.set_yscale("linear")
    ax1.set_xlabel(r"$\log$ $m_{\chi}$[\textrm{GeV}]", fontsize=fontsize)
    ax1.set_ylabel(r"$\log$ $<\sigma \nu>$[$cm^{3} s^{-1}$]", fontsize=fontsize)
    ax1.tick_params(axis = 'both', which = 'major', labelsize=fontsize)
    ax1.tick_params(axis = 'both', which = 'minor', labelsize=fontsize)
    ax1.set_xlim(2.8, max(np.log10(mass_grid)))
    ax1.set_ylim(min(np.log10(sv_grid)), -21)  # 
    #ax1.text(3.5, -23.7, "IceCube Simulation", fontsize=fontsize)
    #ax1.text(3.2, -24.8, r"P-ONE $\nu_\mu$", fontsize=fontsize)
    #ax1.text(3.2, -24.9, r" $\nu_e$", fontsize=fontsize)
    #ax1.text(3.1, -25.3, r" $\nu_\tau$", fontsize=fontsize)
    # ---------------------------------------------
    # Legend
    # plt.legend(fontsize=fontsize)
    h, l = ax1.get_legend_handles_labels()
    lgd1 = ax1.legend(h, l, loc=7, bbox_to_anchor=(1.09, +1.09),
                     ncol=2, fontsize=fontsize, handlelength=h_length,
                     fancybox=True, frameon=False)
    ax1.add_artist(lgd1)
    # legend_elements = [Line2D([0], [0], color=color_sm[j], lw=1),
    #                Line2D([0], [0], linestyle='--', color=color_sm[j],  lw=1),
    #                Line2D([0], [0], linestyle='-.', color=color_sm[j],  lw=1.3),
#                Line2D([0], [0], linestyle=':',  color=color_sm[j], lw=1),
#                #Line2D([0], [0], color=color_sm[j], linestyle='dotted', lw=2),
#               ]
# labels = [r'0.45, 0.25',
#        r'0.35, 0.15',
#        r'0.25, 0.10',
#        r'0.15, 0.09']
# plt.legend(legend_elements, labels)
#h, _ = ax1.get_legend_handles_labels([legend_elements])
#lgd2= ax1.legend(h, labels, loc=7, bbox_to_anchor=(1, 0.8),
#                 ncol=1, fontsize=fontsize, handlelength=h_length,
#                 fancybox=True, frameon=False)
#ax1.add_artist(lgd2)
    plt.grid()
# ---------------------------------------------
# Showing the image
    plt.show()
# ---------------------------------------------
# Storing the image
    figure.savefig(PICS + "Limits_christ_%s.png" %(i[2]),
           bbox_inches='tight', dpi=export_dpi, facecolor="white")


In [ ]:
back_t = pickle.load(open('../data/Back_christ.pkl', 'rb'))
# back_ang = pickle.load(open('../data/Back_christ_ang.pkl', 'rb')) 
atm = Atm_Shower()

In [ ]:
figure, ax1 = plt.subplots(1,1,figsize=(std_size,std_size), sharex=True) 


ax1.step(np.log10(atm._egrid), 2*back_t['numu'], color='b', label=r'P-ONE' )

ax1.set_yscale("log")

ax1.set_xlabel(r'$\log_{10}$ ( $E$ / $ \textrm{GeV} )$ ', fontsize=fontsize)

ax1.set_ylabel(r'\# $Counts$', fontsize=fontsize)

ax1.tick_params(axis = 'both', which = 'major', labelsize=fontsize, direction='in')
ax1.tick_params(axis = 'both', which = 'minor', labelsize=fontsize, direction='in')



ax1.set_xlim(3, 6)

ax1.set_ylim(1,1e6)
h, l = ax1.get_legend_handles_labels()
lgd1 = ax1.legend(h, l, loc=7, bbox_to_anchor=(0.85, +1.03),
                 ncol=2, fontsize=fontsize, handlelength=h_length,
                 fancybox=True, frameon=False)
ax1.add_artist(lgd1)

# ---------------------------------------------
# Showing the image
plt.show()
#figure.savefig(PICS + "Final_eff_area"+".png",
 #              bbox_inches='tight', dpi=export_dpi, facecolor='white')

In [ ]:
# for i,a in enumerate(config['pone_christian']['angles']):
#     figure, ax1 = plt.subplots(1,1,figsize=(std_size,std_size), sharex=True) 
#     ax1.set_title('Zenith angle = %f' % (a))
# 
#     ax1.step(np.log10(atm._egrid), 2*back_ang['numu'][i], color='b', label=r'P-ONE' )
#     
#     ax1.set_yscale("log")
#     
#     ax1.set_xlabel(r'$\log_{10}$ ( $E$ / $ \textrm{GeV} )$ ', fontsize=fontsize)
#     
#     ax1.set_ylabel(r'\# Counts', fontsize=fontsize)
#     
#     ax1.tick_params(axis = 'both', which = 'major', labelsize=fontsize, direction='in')
#     ax1.tick_params(axis = 'both', which = 'minor', labelsize=fontsize, direction='in')
#     
#     
#     
#     
#     ax1.set_xlim(3, 6)
#     #ax1.set_ylim(1,1e10)
#     h, l = ax1.get_legend_handles_labels()
#     lgd1 = ax1.legend(h, l, loc=7, bbox_to_anchor=(0.85, +1.09),
#                      ncol=2, fontsize=fontsize, handlelength=h_length,
#                      fancybox=True, frameon=False)
#     ax1.add_artist(lgd1)
#     
#     # ---------------------------------------------
#     # Showing the image
#     plt.show()
#     #figure.savefig(PICS + "Final_eff_area"+".png",
#      #              bbox_inches='tight', dpi=export_dpi, facecolor='white')
#     break

Sample of file system 

In [ ]:
a_eff_file = pickle.load(open("../data/aeff_cluster_nuecc.pkl",'rb'))
smearing_file = pickle.load(open("../data/fisher_casc.pkl",'rb'))

In [ ]:
a_eff_file

In [ ]:
cos_zen_edges, log10E_edges, aeff_hist = a_eff_file

In [ ]:
np.abs(np.rad2deg(np.arccos(cos_zen_edges)) - 90)

In [ ]:
aeff = aeff_hist.loc[2, 3, 2, 50]["aeff_hist"]

plt.pcolormesh(cos_zen_edges, log10E_edges, aeff.T)

In [ ]:
m_log10E = []
for i,e in enumerate(log10E_edges):
    if i == 0:
        m_log10E.append(log10E_edges[i])
    elif i == 40:
        break
    else:
        m_log10E.append((log10E_edges[i] + log10E_edges[i-1]) / 2)
    

In [ ]:
aeff_hist.index.get_level_values(0)

In [ ]:
aeff_hist.loc[2, 3, 2, 50]["aeff_hist"][0]

In [ ]:
aeff.shape, cos_zen_edges.shape, log10E_edges.shape

In [ ]:
smearing_file.index.get_level_values(1)

In [ ]:
x = 'abc' 

In [ ]:
smearing_file

In [ ]:
"Full pessimistic triang"

In [ ]:
e_bin = np.logspace(2, 6, 21, base=10)
cos_bin = np.linspace(-1,1,20)
energies = smearing_file.loc[("Full pessimistic cluster", 50, slice(None), 16), "logE"].index.get_level_values(2)
log10Esigmas = smearing_file.loc[("Full pessimistic cluster", 50, slice(None), 16), "logE"].values
sp = UnivariateSpline(energies, log10Esigmas )

In [ ]:
smearing_file.loc[("Full pessimistic cluster", 50, energies[2], 16), "logE"]

In [ ]:
energies.shape, log10Esigmas.shape

In [ ]:
log10Esigmas

In [ ]:
smearing_file.loc[("Full pessimistic cluster", 50, slice(None), 16), "logE"].index.get_level_values(2)

In [ ]:
e_bin

In [ ]:
def local_log_normal(x, mu, s):
    """Distribution function 
    x = log10(E_grid)
    mu = log10(E) 
    sigma = ( standard deviation as per definition ) ( so no percentage or fraction )
    """
    s = ( s  )
    pdf = np.exp(-( np.log10(x) - mu )**2 / (2 * s**2)) / (x * s * np.sqrt(2 * np.pi))

    return pdf

In [ ]:
def local_normal(x, mu, s):
    """Gaussian distribution function
    x = E grid
    mu = mean E, log10E
    s = sigma, log10E(standard deviation )
    """
    x = np.log10(x)
    pdf = np.exp(-(((x - mu) / s)**2) / 2) / (s * np.sqrt(2 * np.pi))
    return pdf

In [ ]:
e_grid = atm.egrid

In [ ]:
def find_nearest(array: np.array, value: float):

    """ Returns: index of the nearest vlaue of an array to the given number
    --------------
    idx :  float
    """
    array = np.array(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [ ]:
find_nearest(e_grid, energies[0])

In [ ]:
spl_sigma = UnivariateSpline(energies, log10Esigmas, k=1, s=0, ext=3)
sigmas = spl_sigma(e_grid)

In [ ]:
print('energies_low = %.1e, _high = %.1e' % (e_grid[40], e_grid[67]))

In [ ]:
#for j,s in enumerate(sigmas): 
#    if s < 0.01:
#       sigmas[j] = 0.01

In [ ]:
t_p = 40
e_grid_c = e_grid[t_p:]
e_bin_c = np.log10(e_grid_c)
sigmas_c = sigmas[t_p:]

In [ ]:
grid_recon = []
for i,e in enumerate(e_grid_c):
    tmp_e_recon = local_normal(e_grid_c, np.log10(e), sigmas_c)
    norm = (np.sum(np.nan_to_num(tmp_e_recon)))
    grid_recon.append(np.log10(e) * np.nan_to_num(tmp_e_recon) / norm) 


In [ ]:
grid_recon

In [ ]:
e_bin = np.log10(e_grid_c) #np.log10(energies)
figure, (ax1) = plt.subplots(1, 1, figsize=(std_size, std_size * 6./8. ), sharex=True)
z_p=ax1.imshow((np.array(grid_recon)), origin='lower', extent=( min(e_bin), max(e_bin), min(e_bin), max(e_bin)),
               cmap=plt.get_cmap('bone_r'))

#ax1.contour(np.array(grid_recon).T, levels = [0.68], colors='red', linestyles='-',
#            origin='lower', extent=( min((e_bin)), max((e_bin)), min((e_bin)), max((e_bin))) )
figure.colorbar(z_p, ax=(ax1))
#ax1.contour((samples_fun*e_grid).T, levels = [1.2], colors='purple', linestyles='-',
#            origin='lower', extent=( min(np.log10(e_grid)), max(np.log10(e_grid)), min(np.log10(e_grid)), max(np.log10(e_grid))) )


ax1.set_xlabel(r"$\log_{10}$ $E_{true} / \textrm{GeV}$", fontsize=fontsize)
ax1.set_ylabel(r"$\log_{10}$ $E_{Recon} / \textrm{GeV}$", fontsize=fontsize)
#ax1.set_ylabel(r"Distribution fraction", fontsize=fontsize)
ax1.tick_params(axis = 'both', which = 'major', labelsize=fontsize, direction='in')
ax1.tick_params(axis = 'both', which = 'minor', labelsize=fontsize, direction='in')
ax1.set_xlim(2,7)
ax1.set_ylim(2,7)
# ---------------------------------------------
# Legend
# plt.legend(fontsize=fontsize)
h, l = ax1.get_legend_handles_labels()
lgd1 = ax1.legend(h, l, loc=9, bbox_to_anchor=(0.4, +1.),
                 ncol=1, fontsize=fontsize, handlelength=h_length,
                 fancybox=True, frameon=False)
ax1.add_artist(lgd1)
figure.savefig(PICS + "Reconstruction_"+".png",
               bbox_inches='tight', dpi=export_dpi, facecolor='white')


In [ ]:
smearing_file.columns

In [ ]:
a_eff_file[0].shape, a_eff_file[1].shape

In [ ]:
a_eff_file[2]['aeff_hist'][1, 1, math.nan, 50]

In [ ]:
test_keys_aeff_11 = [[1, 1, 'nan',  50.0], [1, 1, 'nan',  80.0], [1, 1, 'nan',  110.0], [1, 1, 'nan',  140.0], [1, 1, 'nan',  200.0]]
test_keys_aeff_12 = [[1, 2, 'nan', 50.0], [1, 2, 'nan', 80.0], [1, 2, 'nan', 110.0],  [1, 2, 'nan', 140.0], [1, 2, 'nan', 200.0] ]
test_keys_aeff_21 = [[2, 1, 'nan', 50.0], [2, 1, 'nan', 80.0], [2, 1, 'nan', 110.0], [2, 1, 'nan', 140.0],  [2, 1, 'nan', 200.0]]
test_keys_aeff_22 = [[2, 2, 'nan', 50.0],  [2, 2, 'nan', 80.0], [2, 2, 'nan', 110.0], [2, 2, 'nan', 140.0],  [2, 2, 'nan', 200.0]]
spacing = [50, 80, 110, 140, 200]
hit = [1, 2]
module = [1, 2]

In [ ]:
spacing

In [ ]:
for i,k in enumerate(spacing):
    print(np.array(a_eff_file[2]['aeff_hist'][1, 1, math.nan, spacing[i]]).shape)
    print(np.array(a_eff_file[2]['aeff_hist'][1, 2, math.nan, spacing[i]]).shape)
    print(np.array(a_eff_file[2]['aeff_hist'][2, 1, math.nan, spacing[i]]).shape)
    print(np.array(a_eff_file[2]['aeff_hist'][2, 2, math.nan, spacing[i]]).shape)
    